In [1]:
## To do: Write to dos here

In [2]:
# --- Global variables

# Setting up the Coordinate Reference Systems up front in the necessary format.
crs_degree = {'init': 'epsg:4326'} # CGS_WGS_1984 (what the GPS uses)

# --- Paths

# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
# Root path of the Dropbox business account
dbx = get_dropbox_location()

# Temporary! Location of the folder where the restructuring is currently happening
data_path = dbx + '/Private Structured data collection'

In [3]:
import geopandas as gpd
from shapely import geometry

## 1. Creation of the project delimitation

In [4]:
## create the project delimitation

# this code creates the project delimitation from a set of points
p = []
p.append((-121.94277062699996, 37.55273259000006))
p.append((-121.94099807399999, 37.554268507000074))
p.append((-121.91790942699998, 37.549823434000075))
p.append((-121.89348666299998, 37.52770136500004))
p.append((-121.90056572499998, 37.52292299800007))
p.append((-121.90817571699995, 37.52416183400004))
p.append((-121.91252749099999, 37.51845069500007))
p.append((-121.91349347899995, 37.513972023000065))
p.append((-121.90855417099999, 37.503837324000074))
p.append((-121.91358547299996, 37.50097863000008))
p.append((-121.90798018999999, 37.49080413200005))
p.append((-121.91894942199997, 37.48791568200005))
p.append((-121.92029048799998, 37.488706567000065))
p.append((-121.93070953799997, 37.48509600500006))
p.append((-121.93254686299997, 37.48864173700008))
p.append((-121.94079404499996, 37.50416395900004))
p.append((-121.94569804899999, 37.51332606200003))
p.append((-121.94918207899997, 37.520371545000046))
p.append((-121.95305006999996, 37.52804520800004))
p.append((-121.953966735, 37.53272020000003))
p.append((-121.95428756799998, 37.53817435800005))
p.append((-121.95506236799997, 37.54107322100003))
p.append((-121.95676186899999, 37.54656695700004))
p.append((-121.95529950799994, 37.54980786700003))
p.append((-121.95261192399994, 37.550479763000055))
p.append((-121.94988481799999, 37.55277211300006))
p.append((-121.94613010599994, 37.55466923100005))
p.append((-121.94277062699996, 37.55273259000006))

delimitation_poly = geometry.Polygon([[po[0], po[1]] for po in p])

project_del = gpd.GeoDataFrame({'Type': ['Delimitation','Box'], 'geometry': [delimitation_poly, delimitation_poly.envelope]})

lat = [i[0] for i in delimitation_poly.envelope.exterior.coords]
lng = [i[1] for i in delimitation_poly.envelope.exterior.coords]
lat_min = min(lat)
lat_max = max(lat)
lng_min = min(lng)
lng_max = max(lng)
print("Box delimitation: " + str((lat_min, lat_max, lng_min, lng_max)))

project_del.crs = 'epsg:4326'
project_del_path = dbx + "/Private Structured data collection/Manual-made dataset (do not touch)/Network/Map/Project Delimitation/Project_delimitation.shp"
project_del.to_file(project_del_path)

Box delimitation: (-121.95676186899999, -121.89348666299998, 37.48509600500006, 37.55466923100005)


## download the OSM network

See https://github.com/jwass/geopandas_osm
See http://overpass-turbo.eu
See https://automating-gis-processes.github.io/CSC18/course-info/Installing_Anacondas_GIS.html
See https://github.com/gboeing/osmnx-examples/blob/master/notebooks/03-example-osm-place-network.ipynb
See https://automating-gis-processes.github.io/CSC18/lessons/L3/retrieve-osm-data.html


### Add download of public traces + historical network data

### Parse OSM network to check:
- if roads have speed limits, capacity, and cost functions
- if they are traffic lights and stop signs information
- if they are points of interest
- turn restrictions
- historical data (when the road sections has been created)

### Then convert OSM to shapefile and render it with Kepler.gl

In [ ]:
## external centroids with OSM network and closer delimitation

In [ ]:
## Create the external TAZs using Google Maps API

In [ ]:
## Get google maps POIs

In [ ]:
## Get neighborhood information + other shapefile data

In [ ]:
## Do k-mean clustering to create internal centroids

In [ ]:
## Get boundaries of internal TAZs

In [ ]:
## Create TAZs

In [ ]:
## Create centroid connections

In [ ]:
## Create files for Aimsun with centroid connections and centroid and OSM network

See [Aimsun documentation to create the layers](https://www.aimsun.com/tech_notes/gis-import-1/)

In [ ]:
## Download Google Maps tiles corresponding to the project